In [1]:
import cv2
import numpy as np
import pyautogui

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Define the screen width and height
screen_width, screen_height = pyautogui.size()

# Define the hand detection frame region (modify these values based on your preference)
frame_region_start = (100, 100)
frame_region_end = (300, 300)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally (mirror effect)
    frame = cv2.flip(frame, 1)

    # Draw the rectangle for the hand detection region
    cv2.rectangle(frame, frame_region_start, frame_region_end, (255, 0, 0), 2)

    # Extract the region of interest
    roi = frame[frame_region_start[1]:frame_region_end[1], frame_region_start[0]:frame_region_end[0]]

    # Convert ROI to HSV color space (to detect skin color more effectively)
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # Define skin color range in HSV
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)

    # Detect skin color
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the largest contour and its center
        max_contour = max(contours, key=cv2.contourArea)
        M = cv2.moments(max_contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])

            # Map the center of the hand to screen coordinates
            screen_x = np.interp(cx, (0, frame_region_end[0] - frame_region_start[0]), (0, screen_width))
            screen_y = np.interp(cy, (0, frame_region_end[1] - frame_region_start[1]), (0, screen_height))

            # Move the cursor
            pyautogui.moveTo(screen_x, screen_y)

    # Display the frame
    cv2.imshow("Frame", frame)

    # Break the loop with the ESC key
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()

In [4]:
import cv2
import mediapipe as mp
import pyautogui

# Initializing MediaPipe hand tracking module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Get the screen size
screen_width, screen_height = pyautogui.size()

while True:
    success, frame = cap.read()
    if not success:
        break

    # Flip the frame horizontally and convert the color from BGR to RGB
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and detect hands
    results = hands.process(frame_rgb)

    # Check if any hand is detected
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get the tip of the index finger
            tip_x = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x
            tip_y = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y

            # Convert the coordinates to screen resolution
            screen_x = np.interp(tip_x, [0, 1], [0, screen_width])
            screen_y = np.interp(tip_y, [0, 1], [0, screen_height])

            # Move the cursor
            pyautogui.moveTo(screen_x, screen_y)

    # Display the frame
    cv2.imshow("Hand Tracking", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC to break
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
import cv2
import mediapipe as mp
import pyautogui
import numpy as np

# Initializing MediaPipe hand tracking module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Get the screen size
screen_width, screen_height = pyautogui.size()

# Define the detection frame region (modify these values as needed)
# Positioned a bit above the center; adjust based on your preference
frame_region = {
    "start_point": (100, 100),  # Top left corner of the rectangle
    "end_point": (540, 380),    # Bottom right corner of the rectangle
    "color": (255, 0, 0),       # Rectangle color (BGR)
    "thickness": 2,             # Thickness of the rectangle borders
}

while True:
    success, frame = cap.read()
    if not success:
        break

    # Flip the frame horizontally and convert the color from BGR to RGB
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Draw the detection frame region on the webcam feed
    cv2.rectangle(frame, frame_region["start_point"], frame_region["end_point"], frame_region["color"], frame_region["thickness"])

    # Process the frame and detect hands
    results = hands.process(frame_rgb)

    # Check if any hand is detected
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get the tip of the index finger
            tip_x = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x
            tip_y = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y

            # Convert hand landmark position to relative frame coordinates
            relative_x = int(tip_x * frame.shape[1])
            relative_y = int(tip_y * frame.shape[0])

            # Check if the index finger tip is within the detection frame region
            if (frame_region["start_point"][0] < relative_x < frame_region["end_point"][0]) and (frame_region["start_point"][1] < relative_y < frame_region["end_point"][1]):
                # Draw hand landmarks
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Map the hand position to the screen size
                screen_x = np.interp(relative_x, [frame_region["start_point"][0], frame_region["end_point"][0]], [0, screen_width])
                screen_y = np.interp(relative_y, [frame_region["start_point"][1], frame_region["end_point"][1]], [0, screen_height])

                # Move the cursor
                pyautogui.moveTo(screen_x, screen_y)

    # Display the frame
    cv2.imshow("Hand Tracking", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC to break
        break

cap.release()
cv2.destroyAllWindows()

In [6]:
import cv2
import mediapipe as mp
import pyautogui
import numpy as np

# hand tracking module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)

# Get the screen size
screen_width, screen_height = pyautogui.size()

# margins are defined for making the frame region appear at top corner
top_margin = 50
right_margin = 50
frame_width = 300
frame_height = 200

while True:
    success, frame = cap.read()
    if not success:
        break

    # Get the frame dimensions
    frame_height_cam, frame_width_cam, _ = frame.shape

    frame_region = {
        "start_point": (frame_width_cam - frame_width - right_margin, top_margin),
        "end_point": (frame_width_cam - right_margin, top_margin + frame_height),
        "color": (255, 0, 0), 
        "thickness": 1, 
    }

    # Flip the frame horizontally and convert the color from BGR to RGB - generated by GPT because it was having some error
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Draw the detection frame region on the webcam feed
    cv2.rectangle(frame, frame_region["start_point"], frame_region["end_point"], frame_region["color"], frame_region["thickness"])

    # Process the frame and detect hands
    results = hands.process(frame_rgb)

    # Check if any hand is detected
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get the tip of the index finger
            tip_x = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x
            tip_y = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y

            # Convert hand landmark position to relative frame coordinates
            relative_x = int(tip_x * frame_width_cam)
            relative_y = int(tip_y * frame_height_cam)

            # Check if the index finger tip is within the detection frame region
            if (frame_region["start_point"][0] < relative_x < frame_region["end_point"][0]) and (frame_region["start_point"][1] < relative_y < frame_region["end_point"][1]):
                # Draw hand landmarks
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Map the hand position to the screen size
                screen_x = np.interp(relative_x, [frame_region["start_point"][0], frame_region["end_point"][0]], [0, screen_width])
                screen_y = np.interp(relative_y, [frame_region["start_point"][1], frame_region["end_point"][1]], [0, screen_height])

                # Move the cursor
                pyautogui.moveTo(screen_x, screen_y)

    # Display the frame
    cv2.imshow("Hand Tracking", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC to break
        break

cap.release()
cv2.destroyAllWindows()